# Rastermap prototype
* setup 1/27/24
* works with a single session
* the shading for moving/not moving intervals sometimes doesn't work
* 2/11/2024 used this notebook to generate figures for the R01 submission with YP 

In [ ]:
import sys
import json
from os.path import join, dirname
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
from rastermap import Rastermap

sys.path.append("/home/gergely/code/sleep/src")

from classes.suite2p_class import Suite2p as s2p
import classes.imaging_data_class as idc
import classes.behavior_class as bc

### Global variables

In [ ]:
mouse_id = "dock11b1"

In [ ]:
imaging_data = idc.ImagingData(mouse_id=mouse_id)
s2p_folders = [
    (index, folder) for index, folder in enumerate(imaging_data.find_s2p_folders())
]
s2p_folders

In [ ]:
folder_to_load = 3
s2p_folder = s2p_folders[folder_to_load][1]

In [ ]:
s2p_data = s2p(s2p_folder)
spikes = s2p_data.get_spikes(plane=1)
# z scoricng
n_neurons, n_time = spikes.shape
spks = zscore(spikes, axis=1)

In [ ]:
behavior = bc.behaviorData(mouse_id=mouse_id)
behavior_folders = [
    (index, folder) for index, folder in enumerate(behavior.find_behavior_folders())
]
behavior_folders

In [ ]:
beh_folder_to_load = 6
behavior_file = "dock11b1_20240803124357_953_sleep.json"

In [ ]:
processed_behavior = behavior_folders[beh_folder_to_load][1]

if not dirname(processed_behavior) == dirname(s2p_folder):
    raise ValueError("Behavior and imaging data folders do not match")

try:
    with open(join(processed_behavior, behavior_file), "r") as f:
        beh = json.load(f)
except FileNotFoundError:
    print("File not found")

In [ ]:
with open(join(processed_behavior, "filtered_velocity.json"), "r") as f:
    speed = np.array(json.load(f))

position = np.array(beh[0]["data"]["treadmillPosition"])

with open(join(processed_behavior, "mobility_immobility.json"), "r") as f:
    mob_immob = np.array(json.load(f))

In [ ]:
position.shape

### Averaging spikes based on behavior:

In [ ]:
if len(spks[0]) != len(mob_immob):
    print(f"spks.shape, mob_immob.shape)= {spks.shape, mob_immob.shape}, Interpolating")
    x_original = np.linspace(0, 1, len(mob_immob))
    x_target = np.linspace(0, 1, len(spks[0]))
    interpolator = interp1d(x_original, mob_immob, kind="linear")
    mob_immob = interpolator(x_target)
    print(len(mob_immob), len(spks[0]))

In [ ]:
delta_t = 0.033345968  # smapling rate of the imaging data
moving_spks = spks[:, mob_immob == 1].mean()
notmoving_spks = spks[:, mob_immob != 1].mean()

# calculating spike rates:
total_duration_moving = np.sum(mob_immob == 1) * delta_t
total_duration_notmoving = np.sum(mob_immob != 1) * delta_t

moving_rate = moving_spks / total_duration_moving if total_duration_moving > 0 else 0
notmoving_rate = (
    notmoving_spks / total_duration_notmoving if total_duration_notmoving > 0 else 0
)

data = {
    "Condition": ["Moving", "Not Moving"],
    "Mean Value": [moving_spks, notmoving_spks],
    "Mean Rate": [moving_rate, notmoving_rate],
}

df = pd.DataFrame(data)
df["Condition"] = df["Condition"].astype("category")
condition_mapping = {"Moving": 1, "Not Moving": 2}
df["Condition_numeric"] = df["Condition"].map(condition_mapping)
# df.to_json(
#     join(dirname(processed_behavior), "mobility_immobility_mean_spike_rate.json")
# )

In [ ]:
# Convert the data to a long-format DataFrame
df_long = pd.DataFrame(data)
df_long = pd.melt(df_long, id_vars="Condition", var_name="Metric", value_name="Value")

# Use Seaborn's catplot to create point plots
g = sns.catplot(
    x="Condition",
    y="Value",
    col="Metric",
    data=df_long,
    kind="point",
    height=4,
    aspect=1,
    sharey=False,
)

# Adjust the plot aesthetics
g.set_titles("{col_name}")
g.set_axis_labels("", "Value")
# plt.savefig(
#     join(dirname(processed_behavior), "mobility_immobility_mean_and_spike_rate.png")
# )
plt.show()

In [ ]:
# the number of PCs to calculate is half the number of neurons
PCs = int(np.ceil(spks.shape[0] / 2))
PCs

In [ ]:
model = Rastermap(
    n_clusters=3,  # None turns off clustering and sorts single neurons
    n_PCs=PCs,  # use fewer PCs than neurons
    locality=0.1,  # some locality in sorting (this is a value from 0-1)
    time_lag_window=5,  # use future timepoints to compute correlation
    grid_upsample=0,  # 0 turns off upsampling since we're using single neurons
    time_bin=5,
).fit(spks)
y = model.embedding  # neurons x 1
isort = model.isort

compute tuning curve for each neuron

In [ ]:
n_pos = 100
bins = np.arange(0, 1 + 1.0 / n_pos, 1.0 / n_pos)
ibin = np.digitize(position, bins) - 1
n_bins = ibin.max()
inan = np.isnan(position)
ibin[inan] = -1
tcurves = np.zeros((spks.shape[0], n_bins))
for b in range(n_bins):
    tcurves[:, b] = spks[:, ibin == b].mean(axis=1)
tcurves -= tcurves.mean(axis=1, keepdims=True)

In [ ]:
# make figure with grid for easy plotting
from turtle import down

fig = plt.figure(figsize=(12, 8), dpi=200)
grid = plt.GridSpec(10, 24, figure=fig, wspace=0.1, hspace=0.4)
# time = np.linspace(0, 3600, 36000)

# plot location
ax = plt.subplot(grid[0, :-5])
ax.plot(position, lw=1)  # , color=kp_colors[0])
ax.set_xlim([0, len(position)])
ax.axis("off")
ax.set_title("2D location")

# plot running speed
ax = plt.subplot(grid[1, :-5])
ax.plot(speed, color=0.5 * np.ones(3), lw=1)
ax.set_xlim([0, len(speed)])
ax.axis("off")
ax.set_title("running speed")

# Plot spike mean for each condition (moving vs. not moving)
ax = plt.subplot(grid[0:2, -3:])
colors = ["#4c72b0", "#dd8452"]  # Specify colors for each condition
for cond, color in zip(condition_mapping.keys(), colors):
    cond_data = df[df["Condition"] == cond]
    ax.scatter(cond_data["Condition_numeric"], cond_data["Mean Value"], color=color)

# Adjusting plot aesthetics
ax.set_xticks(list(condition_mapping.values()))
ax.set_xticklabels(list(condition_mapping.keys()), fontsize=6)
ax.tick_params(axis="y", labelsize=6)
ax.set_xlim(0.5, 2.5)
ax.set_ylabel("Z-Scored Mean", fontsize=6)
ax.set_title("Spike Mean", fontsize=6)
sns.despine()

# plot sorted neural activity
ax = plt.subplot(grid[2:, :-5])
time_extent = [0, 3600, 0, spks.shape[0]]  # [xmin, xmax, ymin, ymax]
ax.imshow(
    spks[isort], cmap="gray_r", vmin=0, vmax=1.2, aspect="auto", extent=time_extent
)
ax.set_xlabel("time")
ax.set_ylabel("superneurons")

# Shaded mobility periods
down_mobility = np.array(
    [any(mob_immob[i : i + 10]) for i in range(0, len(mob_immob), 10)]
)
mobility = down_mobility.astype(int)
# Assuming x-axis aligns with the length of immobility array
x = np.arange(len(mobility))

# Find where immobility periods start and end
immobility_starts = np.where(np.diff(mobility) == 1)[0] + 1
immobility_ends = np.where(np.diff(mobility) == -1)[0]

# Plot immobility periods
for start, end in zip(immobility_starts, immobility_ends):
    ax.fill_between(x[start:end], 0, spks.shape[0], color="#FA8072", alpha=0.3)

# plot single-neuron tuning curves
ax = plt.subplot(grid[2:, -5:])
x = np.arange(0, n_bins)
dy = 2
# xpad = n_pos / 10
nn = spks.shape[0]
for t in range(len(tcurves)):
    ax.plot(x, tcurves[isort[t], :n_pos] * dy - dy / 2 + nn - t, color="k", lw=0.5)

ax.text(n_pos / 2, -14, "position", ha="center")
ax.text(0, -3, "0")
ax.text(n_pos, -3, "2", ha="right")
ax.set_ylim([0, nn])
ax.axis("off")
plt.show()
plt.tight_layout()

# fig.savefig(join(processed_behavior, "rastermap.svg"), dpi=300)

In [ ]:
# make figure with grid for easy plotting
from turtle import down

fig = plt.figure(figsize=(12, 8), dpi=200)
grid = plt.GridSpec(10, 24, figure=fig, wspace=0.1, hspace=0.4)
# time = np.linspace(0, 3600, 36000)

# plot location
ax = plt.subplot(grid[0, :-5])
ax.plot(position, lw=1)  # , color=kp_colors[0])
ax.set_xlim([0, len(position)])
ax.axis("off")
ax.set_title("2D location")

# plot running speed
ax = plt.subplot(grid[1, :-5])
ax.plot(speed, color=0.5 * np.ones(3), lw=1)
ax.set_xlim([0, len(speed)])
ax.axis("off")
ax.set_title("running speed")

# Plot spike mean for each condition (moving vs. not moving)
ax = plt.subplot(grid[0:2, -3:])
colors = ["#4c72b0", "#dd8452"]  # Specify colors for each condition
for cond, color in zip(condition_mapping.keys(), colors):
    cond_data = df[df["Condition"] == cond]
    ax.scatter(cond_data["Condition_numeric"], cond_data["Mean Value"], color=color)

# Adjusting plot aesthetics
ax.set_xticks(list(condition_mapping.values()))
ax.set_xticklabels(list(condition_mapping.keys()), fontsize=6)
ax.tick_params(axis="y", labelsize=6)
ax.set_xlim(0.5, 2.5)
ax.set_ylabel("Z-Scored Mean", fontsize=6)
ax.set_title("Spike Mean", fontsize=6)
sns.despine()

# plot sorted neural activity
ax = plt.subplot(grid[2:, :-5])
time_extent = [0, 3600, 0, spks.shape[0]]  # [xmin, xmax, ymin, ymax]
ax.imshow(
    spks[isort],
    cmap="gray_r",
    vmin=0,
    vmax=1.2,
    aspect="auto",
    extent=time_extent,
)
ax.set_xlabel("time")
ax.set_ylabel("superneurons")

# Shaded mobility periods
down_mobility = np.array(
    [any(mob_immob[i : i + 10]) for i in range(0, len(mob_immob), 10)]
)
mobility = down_mobility.astype(int)
# Assuming x-axis aligns with the length of immobility array
x = np.arange(len(mobility))

# Find where immobility periods start and end
immobility_starts = np.where(np.diff(mobility) == 1)[0] + 1
immobility_ends = np.where(np.diff(mobility) == -1)[0]

# Plot immobility periods
for start, end in zip(immobility_starts, immobility_ends):
    ax.fill_between(x[start:end], 0, spks.shape[0], color="#FA8072", alpha=0.3)

# plot single-neuron tuning curves
ax = plt.subplot(grid[2:, -5:])
x = np.arange(0, n_bins)
dy = 2
# xpad = n_pos / 10
nn = spks.shape[0]
for t in range(len(tcurves)):
    ax.plot(x, tcurves[isort[t], :n_pos] * dy - dy / 2 + nn - t, color="k", lw=0.5)

ax.text(n_pos / 2, -14, "position", ha="center")
ax.text(0, -3, "0")
ax.text(n_pos, -3, "2", ha="right")
ax.set_ylim([0, nn])
ax.axis("off")
plt.show()
plt.tight_layout()

# fig.savefig(join(processed_behavior, "rastermap.svg"), dpi=300)


In [ ]:
model.embedding[:, 0]
plt.scatter(xpos, ypos, cmap="gist_rainbow", c=y, s=1)